In [0]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, style="whitegrid", palette="pastel", color_codes=True)

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut
import os
import csv

In [2]:
##Download data file, set name

!wget https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
!mv 136_Data_Final.csv\?raw\=true 136_Data_Final.csv

--2019-04-10 22:17:31--  https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
Resolving github.com (github.com)... 192.30.255.112, 192.30.255.113
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-10 22:17:32--  https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-10 22:17:32--  https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv
Resolving raw.githubus

In [29]:
#Import data to dataframe
##Implement Random forest stuff here...
df = pd.read_csv('136_Data_Final.csv')
df.set_index('Subject')
df.head()

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,CP4,P8,C4,TP8,T8,...,FZ,F4,F8,T7,FT7,FC3,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value
0,0,136,1,1,-200,-2.6587,-5.8283,-3.6200,-8.4912,-6.0398,-3.0604,-4.7559,2.8927,-4.3415,-5.1104,...,-8.8225,-3.8681,2.3876,-8.3943,-4.2652,-4.2465,-6.1961,-0.7449,-4.0477,-6.5453,-252.5419,70.5294,13.6076,-66.9374,1.0
1,1,136,1,1,-196,-3.3376,-5.6611,-3.5900,-8.8003,-6.9963,-4.0980,-5.9311,1.7392,-5.2842,-5.5081,...,-8.5898,-3.7891,3.7958,-7.6744,-3.2630,-3.2880,-5.5053,-0.7434,-3.1528,-4.8430,-218.5595,51.3878,25.2795,-68.4394,1.0
2,2,136,1,1,-192,-3.4338,-4.6322,-2.9857,-8.3836,-7.6261,-5.0160,-6.9116,0.6151,-6.4004,-6.1577,...,-7.9451,-3.1924,4.2328,-7.7626,-2.6652,-2.0604,-4.9363,-0.8046,-2.6793,-2.9363,-196.0244,68.5147,18.5178,-64.9081,1.0
3,3,136,1,1,-188,-3.6411,-3.6174,-2.5697,-7.9997,-8.4763,-6.2784,-7.9692,-0.9993,-7.7955,-7.2619,...,-7.0601,-2.4542,3.0931,-7.9620,-2.7516,-1.1900,-5.0391,-0.8101,-2.7765,-1.7893,-191.9580,51.4932,22.3986,-66.2655,1.0
4,4,136,1,1,-184,-3.9098,-3.0866,-2.5453,-8.4382,-9.8238,-8.1725,-8.6853,-3.5078,-9.1693,-8.7561,...,-6.6142,-2.6178,0.8142,-6.9270,-3.0337,-0.9082,-5.5007,-0.6975,-3.0212,-2.2171,-177.3466,41.4210,21.8185,-66.5214,1.0


In [64]:
import random
random_samp_list = []
index_list = []
for i in range(1,201):
  index_list.append(i)


train_samp_list = random.sample(index_list,k=100)
  
test_list = []
test_list = [x for x in index_list if x not in train_samp_list]



#df_train = df['Trial'] == i
print(train_samp_list)
print(test_list)



df_training = random_samp_list

[103, 55, 134, 173, 27, 162, 198, 22, 95, 25, 21, 72, 71, 92, 28, 35, 152, 150, 106, 132, 192, 193, 168, 183, 146, 63, 47, 197, 114, 179, 50, 105, 6, 16, 69, 163, 131, 199, 83, 164, 133, 100, 5, 44, 79, 38, 186, 85, 138, 140, 149, 110, 74, 141, 148, 42, 36, 49, 24, 93, 8, 185, 94, 180, 51, 46, 175, 15, 26, 9, 19, 143, 120, 171, 196, 14, 53, 178, 195, 1, 101, 86, 159, 59, 37, 121, 56, 91, 161, 43, 68, 124, 39, 11, 12, 130, 189, 41, 34, 182]
[2, 3, 4, 7, 10, 13, 17, 18, 20, 23, 29, 30, 31, 32, 33, 40, 45, 48, 52, 54, 57, 58, 60, 61, 62, 64, 65, 66, 67, 70, 73, 75, 76, 77, 78, 80, 81, 82, 84, 87, 88, 89, 90, 96, 97, 98, 99, 102, 104, 107, 108, 109, 111, 112, 113, 115, 116, 117, 118, 119, 122, 123, 125, 126, 127, 128, 129, 135, 136, 137, 139, 142, 144, 145, 147, 151, 153, 154, 155, 156, 157, 158, 160, 165, 166, 167, 169, 170, 172, 174, 176, 177, 181, 184, 187, 188, 190, 191, 194, 200]


In [0]:
df_train = df.drop(df.index[train_samp_list])
df_test = df.drop(df.index[test_list])

    
df_x = df_train.drop(['Subject'], axis=1)
df_y = df_test.drop(['Value', 'Subject'], axis=1)

In [0]:
y_pd = df_y
x_pd = df_x

In [0]:
rfc = RandomForestRegressor(n_estimators=25)
rfc.fit(x_pd, y_pd);

In [79]:
y_pd_predicted = rfc.predict(x_pd)
#formula
#def report(y, y_predicted):
 # conf_mat = confusion_marix(y, y_predicted)
  #report = pd.DataFrame(index=['Value'])
  
print(y_pd_predicted)

[[ 6.3600000e+00  1.0000000e+00  1.0000000e+00 ...  4.7974152e+01
   1.0043224e+01 -5.0006312e+01]
 [ 2.7200000e+00  1.0000000e+00  1.0000000e+00 ...  4.3644084e+01
   2.1888792e+01 -5.9716420e+01]
 [ 4.4800000e+00  1.0000000e+00  1.0000000e+00 ...  3.5249904e+01
   2.9501340e+01 -5.6085556e+01]
 ...
 [ 1.0999668e+05  5.0000000e+00  2.0000000e+02 ... -4.2589896e+01
   2.3513860e+01 -2.2229028e+01]
 [ 1.0999656e+05  5.0000000e+00  2.0000000e+02 ... -4.2921724e+01
   2.2523308e+01 -1.9779648e+01]
 [ 1.0999444e+05  5.0000000e+00  2.0000000e+02 ... -3.3874376e+01
   2.3418512e+01 -1.2274140e+01]]
